## Packages prep

In [ ]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.78.1
    Uninstalling openai-1.78.1:
      Successfully uninstalled openai-1.78.1


In [ ]:
pip install google-search-results pandas

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=64741c20c4dc6b0321b478236984443c81373224a17b0b55c566e17302b77c07
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [ ]:
pip install requests

In [ ]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade openperplex

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time
import json
import csv
import openai
import praw
from serpapi import GoogleSearch
from datetime import datetime
import pytz
local_tz = pytz.timezone("America/Toronto")
import glob
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn3
from openperplex import OpenperplexSync

# Set your API keys
openai.api_key = "your openai api key"
SERPAPI_API_KEY = "your serpapi key"

# Google Searching Data Prep

## Helper functions

In [ ]:
# Define headers for requests
headers = {"User-Agent": "Mozilla/5.0"}

def get_serpapi_urls(query, engine='google', num_results=10):
    """Return a list of website URLs from SERPAPI using the given query."""
    params = {
        "engine": engine,
        "q": query,
        "cc": "CA",
        "api_key": SERPAPI_API_KEY,
        "num": num_results
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    organic_results = results.get("organic_results", [])
    urls = []
    for res in organic_results:
        url = res.get("link", None)
        if url:
            urls.append(url)
    return urls

In [ ]:
def get_cleaned_html(url):
    """Attempt to scrape and clean the HTML from a given URL."""
    try:
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        # Remove script and style tags
        for tag in soup(["style"]):
            tag.decompose()
        # Try to extract main content: first look for <article>
        article = soup.find("article")
        if not article:
            # Fallback: try a div with class "content"
            article = soup.find("div", {"class": "content"})
        if not article:
            # As last resort, use the whole page
            article = soup
        cleaned = article.get_text(separator="\n", strip=True)
        return cleaned
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

In [ ]:
def extract_product_data(cleaned_html):
    """
    Use GPT-4o to extract product ranking data from the cleaned HTML.
    This function calls the model with function_call parameters and returns a dictionary.
    """
    try:
        completion = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are an expert at extracting product rankings and critical reviews from article HTML content. "
                        # "Focus on extracting subjective evaluations, opinions, and direct comparisons between products rather than just product descriptions. "
                        # "Critical reviews should include strengths, weaknesses, pros, and cons mentioned about each product. "
                        # "Ignore neutral descriptions that only state product features without an evaluation. "
                        "If explicit rankings exist, maintain them. If no ranking is mentioned, rank the products in the order they appear in the article. Never leave it blank."
                        "If there are more than 10 products mentioned, only keep the top 10."
                    )

                },
                {"role": "user", "content": cleaned_html}
            ],
            functions=[
                {
                    "name": "parse_product_data",
                    "description": (
                        "Extracts ranked products and their corresponding critical reviews from article HTML content. "
                        "If explicit rankings exist, keep them. If no ranking is given, rank them by the order of appearance. Never leave it blank."
                        "If a product has no review in the article, mark it as 'No review available' instead of leaving it empty."
                    ),
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "products": {
                                "type": "array",
                                "items": {
                                    "type": "object",
                                    "properties": {
                                        "Result Rank": {"type": "integer"},
                                        "Product Title": {"type": "string"},
                                        "Comment": {"type": "string"}
                                    }
                                }
                            }
                        }
                    }
                }
            ],
            function_call={"name": "parse_product_data"}
        )
        func_call = completion.choices[0].message.get("function_call", {})
        arguments_str = func_call.get("arguments", "{}")
        data = json.loads(arguments_str)
        return data.get("products", [])
    except Exception as e:
        print(f"Error extracting product data: {e}")
        return None


In [ ]:
def get_reddit_comments(url):
    """
    Fetch the top 10 comments from a given Reddit URL.
    """
    reddit = praw.Reddit(
        client_id="ZUwpxO018ZJGE4E-dfwazQ",
        client_secret="SUE9d8Q_wr_4N1qcebomwtS293GwJw",
        user_agent="MyRedditScraper/0.1",
        check_for_async=False
    )

    submission = reddit.submission(url=url)
    submission.comments.replace_more(limit=0)
    comments = submission.comments.list()
    # Sort comments by score descending, take top 10.
    top_comments = sorted(comments, key=lambda x: x.score, reverse=True)[:10]
    # Concatenate comment texts with their score.
    comments_text = "\n\n".join([f"Score: {c.score}\nComment: {c.body}" for c in top_comments])
    return comments_text

In [ ]:
def extract_product_data_from_reddit(url):
    """
    Use GPT-4o to extract product ranking data from the Reddit comments.
    This function calls the model with function_call parameters and returns a list of product dictionaries.
    Only products with positive (or at least non-negative) reviews are returned;
    if a product is judged to have a negative review, it is skipped.
    """
    try:
        reddit_comments = get_reddit_comments(url)
        # Call OpenAI to analyze the Reddit comments.
        # Note: The system prompt instructs the model to only return products that are positively reviewed.
        completion = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are an expert at extracting product names, rankings and critical reviews information from aggregated Reddit comments. "
                        "Analyze the following top 10 comments and extract the product ranking, product name, a summary of the critical reviews (Comment). "
                        "Only include products that are positively reviewed. If a product is judged to have a negative review, do not include it in the results. "
                        "Rank products in the order they appear."
                        "If a product has no clear review, mark it as 'No review available'."
                    )
                },
                {"role": "user", "content": reddit_comments}
            ],
            functions=[
                {
                    "name": "parse_product_data",
                    "description": (
                        "Extracts ranked products, and their corresponding critical reviews from Reddit comments. "
                        "Only return products with positive reviews. Otherwise, maintain the order of appearance."
                    ),
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "products": {
                                "type": "array",
                                "items": {
                                    "type": "object",
                                    "properties": {
                                        "Result Rank": {"type": "integer"},
                                        "Product Title": {"type": "string"},
                                        "Comment": {"type": "string"}
                                    }
                                }
                            }
                        }
                    }
                }
            ],
            function_call={"name": "parse_product_data"}
        )
        func_call = completion.choices[0].message.get("function_call", {})
        arguments_str = func_call.get("arguments", "{}")
        data = json.loads(arguments_str)
        return data.get("products", [])
    except Exception as e:
        print(f"Error extracting product data from reddit URL {url}: {e}")
        return None


In [ ]:
def scrape_serpapi_products(query, engine='google', required_count=5, num_results=10, csv_filename="output.csv"):
    """
    Uses SERPAPI to fetch website URLs based on the given query and engine, scrapes them to extract product data,
    and writes the results to a CSV file. Each product record is augmented with:
        - 'Source': the URL from which the data was scraped.
        - 'Source Rank': the ordinal number (1-based) of the SERPAPI URL that produced this data.
        - 'Search Source': the engine name (e.g. "google", "google_shopping", etc.)
        - 'Query': the SERPAPI query string used.
        - 'Timestamp': the timestamp when the query was sent to SERPAPI.

    The final CSV file will contain the following columns:
      - Result Rank
      - Title        (was "Product Title")
      - Description  (was "Comment")
      - Source       (was "Source Link")
      - Source Rank
      - Search Source
      - Query
      - Timestamp

    Parameters:
        query (str): The SERPAPI query string.
        engine (str): The SERPAPI engine to use (e.g. "google", "google_shopping").
        required_count (int): The number of successfully scraped website results to collect.
        num_results (int): The number of URLs to ask SERPAPI for.
        csv_filename (str): The filename for the CSV output.
    """
    from datetime import datetime
    import time, csv

    query_used = query
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Fetch URLs using the specified engine
    urls = get_serpapi_urls(query, engine, num_results=num_results)
    print(f"SERPAPI returned {len(urls)} URLs for engine='{engine}'.")

    results_set = set()

    for i, url in enumerate(urls, start=1):
        # handle Reddit specially
        if "reddit" in url.lower():
            print(f"Processing Reddit URL {i}: {url}")
            products_data = extract_product_data_from_reddit(url)
        else:
            if len(results_set) >= required_count:
                break
            print(f"Processing URL {i}: {url}")
            cleaned_html = get_cleaned_html(url)
            if not cleaned_html or len(cleaned_html) < 100:
                print(f"Unable to extract content from {url}, skipping.")
                continue
            products_data = extract_product_data(cleaned_html)

        if not products_data:
            print(f"Extraction failed for {url}, skipping.")
            continue

        for product in products_data:
            product["Source Link"] = url
            product["Source Rank"] = i
            product["Timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # ensure uniqueness
        hashable = tuple(frozenset(prod.items()) for prod in products_data)
        results_set.add(hashable)
        time.sleep(1)

    if len(results_set) < required_count:
        print("Warning: fewer than required valid results were scraped.")

    # flatten
    all_products = []
    for group in results_set:
        for item in group:
            all_products.append(dict(item))

    # write CSV
    with open(csv_filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([
            "Result Rank",
            "Title",
            "Description",
            "Source",
            "Source Rank",
            "Search Source",
            "Query",
            "Timestamp"
        ])
        for prod in all_products:
            writer.writerow([
                prod.get("Result Rank", "N/A"),
                prod.get("Product Title", "N/A"),   # renamed to Title
                prod.get("Comment", "N/A"),         # renamed to Description
                prod.get("Source Link", "N/A"),     # renamed to Source
                prod.get("Source Rank", "N/A"),
                engine,                             # Search Source comes from the engine parameter
                query_used,
                prod.get("Timestamp", timestamp)
            ])

    print(f"CSV file saved: {csv_filename}")


## Five categories - 10 URLs

In [ ]:
# query = "Best baking ingredients"
# scrape_serpapi_products(query, required_count=10, num_results=20, csv_filename="GoogleSearch_baking_ingredients.csv")

In [ ]:
query = "Top recommended instant coffee in Canada"
scrape_serpapi_products(query, required_count=10, num_results=20, csv_filename="GoogleSearch_instant_coffee.csv")

SERPAPI returned 18 URLs for engine='google'.
Processing Reddit URL 1: https://www.reddit.com/r/BuyCanadian/comments/1ilpd72/best_grocery_store_instant_coffee_brand_made_in/
Processing URL 2: https://www.narcity.com/instant-coffee-comparison-canada
Processing URL 3: https://www.andreaference.com/blog/the-best-instant-coffee
Processing URL 4: https://www.bonappetit.com/story/best-instant-coffee?srsltid=AfmBOooEyIlpse_BETT1nYIYBV8jwfA1ZuCvSxAnTOSgatpHWxXjDzSl
Processing URL 5: https://cornercoffeestore.com/best-instant-coffees-canada/
Unable to extract content from https://cornercoffeestore.com/best-instant-coffees-canada/, skipping.
Processing URL 6: https://www.amazon.ca/best-instant-coffee/s?k=best+instant+coffee
Processing URL 7: https://www.allrecipes.com/longform/best-instant-coffee/
Processing URL 8: https://justuscoffee.com/collections/favourites/products/organic-instant-coffee-canada
Processing URL 9: https://www.thekitchn.com/best-instant-coffee-23632498
Processing URL 10: http

In [ ]:
# query = "Best dog food"
# scrape_serpapi_products(query, required_count=10, num_results=20, csv_filename="GoogleSearch_dog_food.csv")

In [ ]:
# query = "Best cat food"
# scrape_serpapi_products(query, required_count=10, num_results=20, csv_filename="GoogleSearch_cat_food.csv")

In [ ]:
# query = "Best cat litter"
# scrape_serpapi_products(query, required_count=10, num_results=20, csv_filename="GoogleSearch_cat_litter.csv")

## Five categories - 5 URLs

In [ ]:
query = "Best baking ingredients"
scrape_serpapi_products(query, required_count=5, num_results=10, csv_filename="GoogleSearch_baking_ingredients.csv")

SERPAPI returned 8 URLs.
Processing Reddit URL 1: https://www.reddit.com/r/Baking/comments/10mjc7b/what_splurge_baking_ingredients_would_you_love_as/
Processing URL 2: https://www.tasteofhome.com/collection/gourmet-baking-ingredients/?srsltid=AfmBOopTEbMm97qxJ87t95_iwCxCVzM-5UY-aPRMV13kewDLzyoA0Jc9
Processing URL 3: https://shop.kingarthurbaking.com/ingredients?srsltid=AfmBOorhZWoyvkzjNiUGtTcIukPkar1lBRVU_QjIZvUzPiIZ3O7i_Gdh
Unable to extract content from https://shop.kingarthurbaking.com/ingredients?srsltid=AfmBOorhZWoyvkzjNiUGtTcIukPkar1lBRVU_QjIZvUzPiIZ3O7i_Gdh, skipping.
Processing URL 4: https://www.foodnetwork.com/recipes/packages/baking-guide/baking-ingredient-guide
Processing URL 5: https://theepicureantrader.com/collections/baking-ingredients?srsltid=AfmBOoq1FgjgAbshTEPFUbsYAMn3at2xYn-WG1meU2P5SsdHfg4_wO0Q
Processing URL 6: https://www.allrecipes.com/article/essential-baking-ingredients/
CSV file saved: GoogleSearch_baking_ingredients.csv


In [ ]:
query = "Best instant coffee"
scrape_serpapi_products(query, required_count=5, num_results=10, csv_filename="GoogleSearch_instant_coffee.csv")

SERPAPI returned 9 URLs.
Processing URL 1: https://www.bonappetit.com/story/best-instant-coffee?srsltid=AfmBOoofsgNvzRb7b1rw71l-UtiXOXoV4ullwoSas0Jt_5L__pcnk3ey
Processing URL 2: https://www.tastingtable.com/1755382/instant-coffee-brands-ranked-worst-best/
Processing Reddit URL 3: https://www.reddit.com/r/Coffee/comments/y9cjhe/are_there_any_good_instant_coffees/
Processing URL 4: https://www.nytimes.com/wirecutter/reviews/best-instant-coffee/
Processing URL 5: https://www.outdoorlife.com/gear/best-instant-coffee-for-backpacking/
CSV file saved: GoogleSearch_instant_coffee.csv


In [ ]:
query = "Best dog food"
scrape_serpapi_products(query, required_count=5, num_results=10, csv_filename="GoogleSearch_dog_food.csv")

SERPAPI returned 7 URLs.
Processing URL 1: https://www.dogfoodadvisor.com/
Processing URL 2: https://www.petmd.com/dog/vet-verified/best-dog-food
Processing URL 3: https://www.nytimes.com/wirecutter/reviews/how-to-buy-the-best-dog-food/
Processing Reddit URL 4: https://www.reddit.com/r/dogs/comments/1hxb7fu/any_good_dog_food_recommendations/
Processing URL 5: https://www.consumerreports.org/health/pet-food/whats-really-in-your-dogs-food-a1115304393/
CSV file saved: GoogleSearch_dog_food.csv


In [ ]:
query = "Best cat food"
scrape_serpapi_products(query, required_count=5, num_results=10, csv_filename="GoogleSearch_cat_food.csv")

SERPAPI returned 10 URLs.
Processing URL 1: https://cats.com/cat-food-reviews
Processing URL 2: https://www.chewy.com/b/food-387
Unable to extract content from https://www.chewy.com/b/food-387, skipping.
Processing URL 3: https://www.forbes.com/sites/forbes-personal-shopper/article/best-cat-foods/
Processing URL 4: https://www.petmd.com/cat/vet-verified/best-cat-food
Processing Reddit URL 5: https://www.reddit.com/r/CatAdvice/comments/1cv5j1r/what_is_the_best_cat_food_price_isnt_a_concern/
Processing URL 6: https://www.vet.cornell.edu/departments-centers-and-institutes/cornell-feline-health-center/health-information/feline-health-topics/feeding-your-cat
CSV file saved: GoogleSearch_cat_food.csv


In [ ]:
query = "Best cat litter"
scrape_serpapi_products(query, required_count=5, num_results=10, csv_filename="GoogleSearch_cat_litter.csv")

SERPAPI returned 8 URLs.
Processing URL 1: https://www.nytimes.com/wirecutter/reviews/best-cat-litter/
Processing Reddit URL 2: https://www.reddit.com/r/CatAdvice/comments/1cmj2l0/whats_the_best_cat_litter/
Processing URL 3: https://www.consumerreports.org/home-garden/best-cat-litter-a1408378942/
Processing URL 4: https://www.amazon.com/Worlds-Best-Cat-Litter-Scoopable/dp/B007ZPX2NA
Unable to extract content from https://www.amazon.com/Worlds-Best-Cat-Litter-Scoopable/dp/B007ZPX2NA, skipping.
Processing URL 5: https://www.thesprucepets.com/best-cat-litters-4154326
Processing URL 6: https://www.worldsbestcatlitter.com/
CSV file saved: GoogleSearch_cat_litter.csv


## Merging datasets of the five categories into one

In [ ]:
csv_files = glob.glob("GoogleSearch_*.csv")

df_list = []

for file in csv_files:
    df = pd.read_csv(file)

    # Fill empty Result Rank using the original DataFrame's index
    df["Result Rank"] = df.groupby("Source")["Result Rank"].transform(
        lambda x: x.fillna(pd.Series(range(1, len(x) + 1), index=x.index))
    )

    # Remove prefix and suffix, then replace underscores with spaces.
    category_name = file.replace("GoogleSearch_", "").replace(".csv", "").replace("_", " ")
    df["Category"] = category_name
    df_list.append(df)

merged_df = pd.concat(df_list, ignore_index=True)

merged_df.to_csv("GoogleSearch_Data_4_15.csv", index=False)

print("All CSV files have been merged into 'GoogleSearch_Data_4_15.csv' with 'Category' column.")

All CSV files have been merged into 'GoogleSearch_Data_4_15.csv' with 'Category' column.


## Instant Coffee - Google and Bing

In [ ]:
query = "Top recommended instant coffee in Canada"
scrape_serpapi_products(query, required_count=10, num_results=20, csv_filename="GoogleSearch_instant_coffee.csv")

SERPAPI returned 18 URLs for engine='google'.
Processing Reddit URL 1: https://www.reddit.com/r/BuyCanadian/comments/1ilpd72/best_grocery_store_instant_coffee_brand_made_in/
Processing URL 2: https://www.narcity.com/instant-coffee-comparison-canada
Processing URL 3: https://www.andreaference.com/blog/the-best-instant-coffee
Processing URL 4: https://www.bonappetit.com/story/best-instant-coffee?srsltid=AfmBOooEyIlpse_BETT1nYIYBV8jwfA1ZuCvSxAnTOSgatpHWxXjDzSl
Processing URL 5: https://cornercoffeestore.com/best-instant-coffees-canada/
Unable to extract content from https://cornercoffeestore.com/best-instant-coffees-canada/, skipping.
Processing URL 6: https://www.amazon.ca/best-instant-coffee/s?k=best+instant+coffee
Processing URL 7: https://www.allrecipes.com/longform/best-instant-coffee/
Processing URL 8: https://justuscoffee.com/collections/favourites/products/organic-instant-coffee-canada
Processing URL 9: https://www.thekitchn.com/best-instant-coffee-23632498
Processing URL 10: http

In [ ]:
scrape_serpapi_products(query, engine='bing',required_count=10, num_results=20, csv_filename="BingSearch_instant_coffee.csv")

SERPAPI returned 10 URLs for engine='bing'.
Processing URL 1: https://cornercoffeestore.com/best-instant-coffees-canada/
Unable to extract content from https://cornercoffeestore.com/best-instant-coffees-canada/, skipping.
Processing URL 2: https://www.dropmocha.ca/post/top-7-instant-coffee-brands-you-need-to-try-in-canada
Processing URL 3: https://www.wired.com/gallery/best-instant-coffee/
Processing URL 4: https://www.dropmocha.ca/post/savoring-convenience-the-best-instant-coffees-to-buy-in-canada
Processing URL 5: https://www.narcity.com/instant-coffee-comparison-canada
Processing URL 6: https://ca.bestreviews.guide/instant-coffees
Processing URL 7: https://www.bestproductscanada.com/instant-coffee
Processing URL 8: https://grindthosebeans.com/coffee-brand-in-canada/
Processing URL 9: https://www.bestratedincanada.com/best-canadian-coffee-brands/
Processing URL 10: https://www.homegrounds.co/best-instant-coffee/
CSV file saved: BingSearch_instant_coffee.csv


In [ ]:
csv_files = glob.glob("*_instant_coffee.csv")

df_list = []

for file in csv_files:
    df = pd.read_csv(file)

    # Fill empty Result Rank using the original DataFrame's index
    df["Result Rank"] = df.groupby("Source")["Result Rank"].transform(
        lambda x: x.fillna(pd.Series(range(1, len(x) + 1), index=x.index))
    )

    # Remove prefix and suffix, then replace underscores with spaces.
    category_name = 'instant coffee'
    df["Category"] = category_name
    df_list.append(df)

SerpAPI_df = pd.concat(df_list, ignore_index=True)

SerpAPI_df.to_csv("SerpAPI_Data_4_21.csv", index=False)

print("All CSV files have been merged into 'SerpAPI_Data_4_21.csv'.")

All CSV files have been merged into 'SerpAPI_Data_4_21.csv'.
